In [2]:
import kfp
import kfp.dsl as dsl
import kfp.onprem as onprem
import kfp.components as comp

# for tensorboard
from kubernetes.client.models import V1EnvVar
    
@dsl.pipeline(
    name='dataPipeline',
    description='data download and train'
)

def data_pipeline():

    admin_pvc = dsl.PipelineVolume(pvc="admin-pvc", name="admin-pvc") # admin-pvc 활용
         
    data_pv = dsl.ContainerOp(
        name='data_download',
        image='kubeflow-registry.default.svc.cluster.local:30000/datadownload-job:9A1FC451',  # fairing image
        command=['python', '/app/01-data-download.py'],
        pvolumes={"/mnt/data" : admin_pvc}                            # folder 명 맞추어야 함
    )
    
    pv_train = dsl.ContainerOp(
        name='mnist_training',
        image='kubeflow-registry.default.svc.cluster.local:30000/traindata-job:7A662BD7',     # fairing image
        command=['python', '/app/02-train-data.py'],
        pvolumes={"/mnt/data" : admin_pvc}                            # folder 명 맞추어야 함
    )
    
    pv_train.after(data_pv) # pipeline code

if __name__ == '__main__':
    my_run = kfp.Client().create_run_from_pipeline_func(data_pipeline, arguments={}, experiment_name='data-pipeline')